In [ ]:
import os
import sys
import numpy as np
#import torch
#import shutil
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import random
#from albumentations.pytorch import ToTensorV2
#import collections
from tqdm import tqdm
import json
import matplotlib.pyplot as plt
from bi_gauss_update import gauss_map_2,save_gauss_2,bi_var_gauss

In [ ]:
train_per = 0.8
val_per = 0.2*train_per

In [ ]:
raw_radar_dir = r'G:\EFS-GX6\4130_Arbeitsgruppen\4137_KHO_Performance\Studentische_Themen\RaviKothari\Dataset\CARRADA_old\Carrada\raw_radar'
radar_folder_list = os.listdir(raw_radar_dir)
random.shuffle(radar_folder_list)
gauss_dir = r'G:\EFS-GX6\4130_Arbeitsgruppen\4137_KHO_Performance\Studentische_Themen\RaviKothari\Dataset\CARRADA_old\50_Dataset\gauss_vis'
doppler_dir = r'G:\EFS-GX6\4130_Arbeitsgruppen\4137_KHO_Performance\Studentische_Themen\RaviKothari\Dataset\datasets_master\maps'

In [ ]:
save_dir = r'G:\EFS-GX6\4130_Arbeitsgruppen\4137_KHO_Performance\Studentische_Themen\RaviKothari\Dataset\CARRADA_old\50_Dataset\bi_variate_norm_latest'

In [ ]:
save_map =r'G:\EFS-GX6\4130_Arbeitsgruppen\4137_KHO_Performance\Studentische_Themen\RaviKothari\Dataset\CARRADA_old\50_Dataset\compare_old_new'

In [ ]:
mask_dir = r'G:\EFS-GX6\4130_Arbeitsgruppen\4137_KHO_Performance\Studentische_Themen\RaviKothari\Dataset\CARRADA_old\50_Dataset\bi_variate_norm_latest\val_mask'
save_dir_n = r'G:\EFS-GX6\4130_Arbeitsgruppen\4137_KHO_Performance\Studentische_Themen\RaviKothari\Dataset\CARRADA_old\50_Dataset\bi_variate_norm_latest\val_ra_map'

In [ ]:
annotate = np.load('gt_annotation.npy',allow_pickle=True).item()

In [ ]:
vect = np.zeros((2,9,9))
for i in range(9):
    for j in range(9):
        vect[0,i,j] = 4-i
        vect[1,i,j] = 4-j

In [ ]:

for frame_name in tqdm(os.listdir(mask_dir)):
    seq,frame,_ = frame_name.split('_')
    map_0 = np.load(os.path.join(raw_radar_dir,seq,'range_angle_numpy',frame+'.npy'))
    map_1 = np.load(os.path.join(raw_radar_dir,seq,'range_angle_numpy',str(int(frame)-1).zfill(6)+'.npy'))
    map_2 = np.load(os.path.join(raw_radar_dir,seq,'range_angle_numpy',str(int(frame)-2).zfill(6)+'.npy'))
    map_3 = np.load(os.path.join(raw_radar_dir,seq,'range_angle_numpy',str(int(frame)-3).zfill(6)+'.npy'))
    map_4 = np.load(os.path.join(raw_radar_dir,seq,'range_angle_numpy',str(int(frame)-4).zfill(6)+'.npy'))
    stacked_map = np.stack([map_0,map_1,map_2,map_3,map_4],axis=0)
    np.save(os.path.join(save_dir_n,seq+'_'+frame + '.npy'),stacked_map)



In [ ]:
for seq in tqdm(radar_folder_list):
    #print(seq)
    
    tracks = annotate[seq]['tracks']
    random.shuffle(tracks)
    train_num = int(len(tracks)*train_per)
    val_num = int(len(tracks)*val_per+1)
    for cnt, track in enumerate(tracks) :
        frame = str(track[-1]).zfill(6)
        ra_map = np.load(os.path.join(raw_radar_dir,seq,'range_angle_numpy',frame+'.npy'))
        rd_map = np.load(os.path.join(doppler_dir,seq,'rd_map',frame+'.npy'))
        ad_map = np.load(os.path.join(doppler_dir,seq,'ad_map',frame+'.npy'))
        ra_mask,center_map,orent_map = bi_var_gauss(annotate[seq][frame],vect)
        
        
        
        if cnt<train_num:
            string = 'train'
        else: string = 'val'
        np.save(os.path.join(save_dir,string +'_ra_map',seq+'_'+frame + '.npy'),ra_map)
        np.save(os.path.join(save_dir,string +'_rd_map',seq+'_'+frame + '.npy'),rd_map)
        np.save(os.path.join(save_dir,string + '_ad_map',seq+'_'+frame + '.npy'),ad_map)
        np.save(os.path.join(save_dir,string +'_center',seq+'_'+frame + '_center.npy'),center_map)
        np.save(os.path.join(save_dir,string +'_mask',seq+'_'+frame + '_mask.npy'),ra_mask)
        np.save(os.path.join(save_dir,string +'_orent',seq+'_'+frame + '_orent.npy'), orent_map)

In [ ]:
count=0
for name in tqdm(os.listdir(val_mask)):
    seq,frame,_= name.split('_')
    
    box_old  = json.load(open(os.path.join(raw_radar_dir,seq,'box_proj.json')))
    box_new = []
    try:
        box_new = np.load(os.path.join(raw_radar_dir,seq,'box_proj.npy'),allow_pickle=True).item()
    except : pass

    if frame in box_old and frame in box_new:
        if len(box_old[frame])!=0 and len(box_new[frame])!=0:
            case = False
            if len(box_old[frame]['idx'])!=len(box_new[frame]['idx']):
                case= True
                count+=1
            
            else:
                for cnt,cl in enumerate(box_new[frame]['idx']):
                    a1_n,r1_n,a2_n,r2_n = box_new[frame]['idx'][cnt]
                    a1,r1,a2,r2 = box_old[frame]['idx'][cnt]
                    if a1_n!=a1 or a2_n!=a2 or r1_n!=r1 or r2_n!=r2: case=True
            if case:
                
                ra_map = np.load(os.path.join(raw_radar_dir,seq,'range_angle_numpy',f"{frame}.npy"))
                ra_mask = np.zeros((3,256,256))
                ra_mask, ra_gauss = gauss_map_2(ra_map,box_new[frame],ra_mask)
                ra_gauss = np.float32(ra_gauss)
                mask_save = os.path.join(val_mask_updated,f"{seq}_{str(frame)}_mask.npy")
                np.save(mask_save,ra_gauss)


                           
                            

print(seq, count)

In [ ]:
count=0
for name in tqdm(os.listdir(val_mask)):
    
    seq,frame,_= name.split('_')
    box_old  = json.load(open(os.path.join(raw_radar_dir,seq,'box_proj.json')))
    box_new = []
    try:
        box_new = np.load(os.path.join(raw_radar_dir,seq,'box_proj.npy'),allow_pickle=True).item()
    except : pass

    if frame in box_old and frame in box_new:
        if len(box_old[frame])!=0 and len(box_new[frame])!=0:
            case = False
            if len(box_old[frame]['idx'])!=len(box_new[frame]['idx']):
                case= True
            
            else:
                for cnt,cl in enumerate(box_new[frame]['idx']):
                    a1_n,r1_n,a2_n,r2_n = box_new[frame]['idx'][cnt]
                    a1,r1,a2,r2 = box_old[frame]['idx'][cnt]
                    if a1_n!=a1 or a2_n!=a2 or r1_n!=r1 or r2_n!=r2: case=True
            if case:
                
                ra_map = np.load(os.path.join(raw_radar_dir,seq,'range_angle_numpy',f"{frame}.npy"))
                rd_map = np.load(os.path.join(val_rd_map,seq+'_'+frame+'.npy'))
                ad_map = np.load(os.path.join(val_ad_map,seq+'_'+frame+'.npy'))
                ra_mask = np.zeros((3,256,256))
                ra_mask, ra_gauss = gauss_map_2(ra_map,box_old[frame],ra_mask)
                fig,ax = plt.subplots(3,3,figsize=(15,15))
                for i in range(3):
                    ax[0,i].imshow(ra_gauss[i,::])
                ra_mask = np.zeros((3,256,256))
                ra_mask, ra_gauss = gauss_map_2(ra_map,box_new[frame],ra_mask)
                for i in range(3):
                    ax[1,i].imshow(ra_gauss[i,::])
                ax[2,0].imshow(ra_map)
                ax[2,1].imshow(rd_map)
                ax[2,2].imshow(ad_map)
                mask_save = os.path.join(save_map,f"{seq}_{str(frame)}.jpg")
                plt.savefig(mask_save)
                plt.close()
                

                


In [ ]:
print(seq,frame[idx])

In [ ]:
# shutil.rmtree(train_mask)
# shutil.rmtree(val_mask)
# shutil.rmtree(train_radar)
# shutil.rmtree(val_radar)

In [ ]:
for counter, seq in enumerate(tqdm(radar_folder_list)):
    #os.mkdir(os.path.join(raw_radar_dir,seq,'gauss'))
    frame = os.listdir(os.path.join(raw_radar_dir,seq, 'annotations','dense'))
    box = json.load(open(os.path.join(raw_radar_dir,seq,'box_proj.json')))
    for idx in range(len(frame)):
        if frame[idx] in box:
            if len(box[frame[idx]])!=0:
                ra_map = np.load(os.path.join(raw_radar_dir,seq,'range_angle_numpy',f"{frame[idx]}.npy"))
                ra_mask = np.zeros((3,256,256))
                ra_mask, ra_gauss = gauss_map_2(ra_map,box[frame[idx]],ra_mask)
                mask_save = os.path.join(raw_radar_dir,seq,'gauss',f"{str(frame[idx])}_gauss.npy")
                np.save(mask_save,ra_gauss)

In [ ]:
print(seq)

In [ ]:
for counter, seq in enumerate(tqdm(radar_folder_list)):
    frame = os.listdir(os.path.join(raw_radar_dir,seq, 'annotations','dense'))
    random.shuffle(frame)
    box = json.load(open(os.path.join(raw_radar_dir,seq,'box_proj.json')))
    for idx in range(15):
        if frame[idx] in box:
            if len(box[frame[idx]])!=0:
                ra_map = np.load(os.path.join(raw_radar_dir,seq,'range_angle_numpy',f"{frame[idx]}.npy"))
                ra_mask = np.zeros((3,256,256))
                ra_mask, ra_gauss = gauss_map_2(ra_map,box[frame[idx]],ra_mask)
                mask_save = os.path.join(gauss_dir,f"{seq}_{str(frame[idx])}_gauss.jpg")
                img_dir = os.path.join(raw_radar_dir,seq,'camera_images',frame[idx]+str('.jpg'))
                save_gauss_2(ra_mask,ra_gauss,ra_map,img_dir,mask_save)


In [ ]:
np.max(ra_gauss)

In [ ]:
for counter, seq in enumerate(tqdm(radar_folder_list)):
    frame = os.listdir(os.path.join(raw_radar_dir,seq, 'annotations','dense'))
    random.shuffle(frame)
    box = json.load(open(os.path.join(raw_radar_dir,seq,'box_proj.json')))
    train_num = int(len(frame)*train_per)
    val_num = int(len(frame)*val_per+1)
    for idx in range(train_num+val_num):
        if frame[idx] in box:
            if len(box[frame[idx]])!=0:
                ra_map = np.load(os.path.join(raw_radar_dir,seq,'range_angle_numpy',f"{frame[idx]}.npy"))
                ra_mask = np.load(os.path.join(raw_radar_dir,seq,'gauss',f"{frame[idx]}_gauss.npy"))
                if idx < train_num:
                    map_save = os.path.join(train_radar,f"{seq}_{str(frame[idx])}.npy")
                    mask_save = os.path.join(train_mask,f"{seq}_{str(frame[idx])}_mask.npy")
                else:
                    map_save = os.path.join(val_radar,f"{seq}_{str(frame[idx])}.npy")
                    mask_save = os.path.join(val_mask,f"{seq}_{str(frame[idx])}_mask.npy")
                
                ra_mask = np.float32(ra_mask)
                ra_map = np.float32(ra_map)
                np.save(map_save,ra_map)
                np.save(mask_save,ra_mask)
            

        

In [ ]:
print(seq)

In [ ]:
mask = np.load(os.path.join(train_mask,seq+str('_000099_mask.npy')))

In [ ]:
plt.imshow(mask[2,:,:])